<a href="https://colab.research.google.com/github/LeoSotoG/Auto_ML_Classification/blob/main/Auto_ML_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Proyect**

El conjunto de datos contiene información sobre clientes bancarios que abandonaron el banco o continúan siendo clientes. Y se planteara un modelo  de rotación de clientes. El modelo de predicción de rotación de clientes bancarios se utiliza para prever qué clientes tienen más probabilidades de abandonar el banco en el futuro.  

##Column Information
El conjunto de datos incluye los siguientes atributos:

* **Customer ID:** Un identificador único para cada cliente.
* **Surname:** El apellido o apellido del cliente.
* **Credit Score:** Un valor numérico que representa la puntuación crediticia del cliente.
* **Geography:** El país donde reside el cliente (Francia, España o Alemania).
* **Gender:** El género del cliente (Masculino o Femenino).
* **Age:** La edad del cliente.
* **Tenure:** El número de años que el cliente ha estado con el banco.
* **Balance:** El saldo de la cuenta del cliente.
* **NumOfProducts:** El número de productos bancarios que utiliza el cliente (por ejemplo, cuenta de ahorro, tarjeta de crédito).
* **HasCrCard:** Si el cliente tiene una tarjeta de crédito (1 = sí, 0 = no).
* **IsActiveMember:** Si el cliente es un miembro activo (1 = sí, 0 = no).
* **EstimatedSalary:** El salario estimado del cliente.
* **Exited:** Si el cliente ha abandonado (1 = sí, 0 = no).

##Load libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Para imputación
from sklearn.impute import SimpleImputer
from scipy.stats import ks_2samp
from scipy.stats import chi2_contingency

# Para el preprocesamiento
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, QuantileTransformer

# Para modelado
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Medir tiempo de ejecución
import time

# Configurar pandas para mostrar números en formato estándar
pd.options.display.float_format = "{:.2f}".format

##Load Data

In [ ]:
data = pd.read_csv("Churn_Modelling.csv")

##Data processing

* **Customer ID:** Si bien es un identificador único para cada cliente, no proporciona información significativa sobre el comportamiento del cliente ni sobre los factores que podrían influir en su decisión de abandonar el banco. Por lo tanto, incluirlo en el modelo probablemente no mejorarí­a su capacidad predictiva y podría incluso introducir ruido en los datos.
* **Surname:** El apellido del cliente tampoco suele ser relevante para predecir la rotación de clientes. Al igual que el ID del cliente, el apellido no proporciona información directa sobre las características o el comportamiento del cliente que podrían estar relacionadas con la rotación. Por lo tanto, tampoco suele incluirse en el modelo.

###Split variables

In [ ]:
categorical_features = ["Geography", "Gender"]

In [ ]:
numerical_features = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary"]

###Imputation of null data

**Variables nulas categoricas (Prueba CHI Cuadrada)**

In [ ]:
# Variables a imputar categoricas
vc_nulas = ["Geography"]

In [ ]:
# Definir imputador por moda
imputer = SimpleImputer(strategy = "most_frequent")

(Prueba CHI Cuadrada)

In [ ]:
for column in vc_nulas:
  X = data[[column]]
  Xi = pd.DataFrame(imputer.fit_transform(X), columns = [column])
  # Creamos una tabla de contingencia
  tabla_contingencia = pd.crosstab(X[column].dropna(), Xi[column])
  # Aplicamos la prueba de Chi-cuadrado
  chi2, p_valor, _, _ = chi2_contingency(tabla_contingencia)
  #Decision
  if p_valor < 0.05:
    print(f"La imputación en la columna {column} fue correcta")
    # Haciendo la imputacion en el DataFrame original
    imputed_values = imputer.transform(data[[column]].copy())
    imputed_df = pd.DataFrame(imputed_values, columns=[column], index=data.index)
    data[column] = imputed_df
  else:
    print(f"La imputación en la columna {column} NO fue correcta")

La imputación en la columna Geography fue correcta


**Variables nulas numericas (Prueba de Kolmogorov-Smirnov)**

In [ ]:
# Variable a imputar numericas
vn_nulas = ["Age", "HasCrCard","IsActiveMember"]

In [ ]:
# Definir imputador por promedio
im = SimpleImputer(strategy = "mean")

(Prueba de Kolmogorov-Smirnov)

In [ ]:
for col in vn_nulas:
  # Descargando la columna
  X = data[[col]]
  # Imputando los datos
  Xi = pd.DataFrame(im.fit_transform(X), columns=[col])
  #Estadistico de prueba
  if ks_2samp(X[col].dropna(),Xi[col]).statistic < 0.1:
    print(f"La imputación en la columna {col} fue correcta")
    # Haciendo la imputacion en el DataFrame original
    data[col] = im.transform(data[[col]].copy())
  else:
    print(f"La imputación en la columna {col} NO fue correcta")

La imputación en la columna Age fue correcta
La imputación en la columna HasCrCard fue correcta
La imputación en la columna IsActiveMember fue correcta


In [ ]:
data["Age"].isnull().sum()

0

##Outliers

###IQR

In [ ]:
# Variables a imputar
var_outliers = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "EstimatedSalary"]

numerical_data = data[numerical_features]

In [ ]:
# Quantile 1 y 3
Q1 = numerical_data.quantile(0.25)
Q3 = numerical_data.quantile(0.75)

# Obteniendo el IQR
IQR = Q3 - Q1

# Inicializar una máscara para todas las filas
mask_total = pd.Series(True, index=data.index)

# Calcular y combinar las máscaras para cada columna
for col in var_outliers:
    mask = ~((numerical_data[col] < (Q1[col] - 1.5 * IQR[col])) | (numerical_data[col] > (Q3[col] + 1.5 * IQR[col])))
    # Alinear las máscaras para asegurarse de que tengan el mismo índice
    mask = mask.reindex(mask_total.index, fill_value=False)
    # Combinar las máscaras
    mask_total &= mask

len(numerical_data), len(numerical_data[mask_total])
# Se perdieron 432 datos

(10002, 9570)

In [ ]:
# Aplicando estos cambios
data = data[mask_total]
data.shape

(9570, 14)

##Pipeline

**Funcion para buscar los hiperparametros de los modelos y encontrar los optimos, utiliza programación paralela.**

In [ ]:
def entrenar(param, modelo, X, y):
    # Inicia una búsqueda aleatoria de hiperparámetros
    grid = RandomizedSearchCV(param_distributions=param,
                              # Utiliza todos los núcleos disponibles para procesamiento paralelo
                              n_jobs=-1,
                              # Número de iteraciones de búsqueda aleatoria
                              n_iter=10,
                              # Número de divisiones para la validación cruzada
                              cv=4,
                              # Estimador del modelo a utilizar
                              estimator=modelo,
                              # Cómo manejar errores durante el ajuste del modelo
                              error_score='raise')

    # Ajusta el modelo utilizando búsqueda aleatoria de hiperparámetros
    grid.fit(X, y)

    # Retorna los resultados de la búsqueda, el mejor estimador, el mejor puntaje, y los mejores parámetros
    return grid, grid.best_estimator_, grid.best_score_, grid.best_params_

**Funcion para evaluar el modelo**

In [ ]:
def metricas(Xt, Xv, yt, yv, modelo):
    # Define una función llamada metricas que toma como entrada los conjuntos de datos de entrenamiento (Xt, yt),
    # los conjuntos de datos de validación (Xv, yv) y el modelo entrenado (modelo).

    d = {'train': round(roc_auc_score(y_true=yt, y_score=modelo.predict_proba(Xt)[:,1]), 3),
         # Calcula el área bajo la curva ROC (AUC) para el conjunto de entrenamiento
         # y lo almacena en un diccionario con la clave 'train'.
         # Utiliza predict_proba para obtener las probabilidades predichas y selecciona la columna correspondiente
         # al valor positivo (columna 1) para calcular el AUC.

         'validate': round(roc_auc_score(y_true=yv, y_score=modelo.predict_proba(Xv)[:,1]), 3)
         # Calcula el área bajo la curva ROC (AUC) para el conjunto de validación
         # y lo almacena en un diccionario con la clave 'validate'.
         # Utiliza predict_proba para obtener las probabilidades predichas y selecciona la columna correspondiente
         # al valor positivo (columna 1) para calcular el AUC.
        }
    return d
    # Retorna el diccionario que contiene los valores de AUC para los conjuntos de entrenamiento y validación.

In [ ]:
# Descargando las variables numericas
numerical_data = data[numerical_features]

# Descargando las variables categoricas
categorical_data = data[categorical_features]

In [ ]:
# Definiendo la target
y = data["Exited"]

###Hiperarametros

In [ ]:
# Hiperarametros del modelo de Regresión Logistica
param_logreg = dict(penalty = ['l1', 'l2'],  # Define la norma utilizada en la regularización: l1 (valor absoluto) y l2 (cuadrado)
                     C = np.arange(0.1, 2, 0.1),  # Define una lista de valores para el parámetro de regularización C, que va desde 0.1 hasta 2 en incrementos de 0.1
                     solver = ['liblinear', 'saga'],  # Define los solvers utilizados para optimizar la función de coste: 'liblinear' para problemas pequeños y 'saga' para problemas grandes
                     max_iter = [100, 200, 300],  # Define el número máximo de iteraciones
                     random_state = [42]  # Define una semilla aleatoria para reproducibilidad
                    )

In [ ]:
# Hiperarametros del modelo Arbol de Decision
param_arbol_decision = dict(
    criterion = ['gini', 'entropy'],  # Criterio para medir la calidad de una división
    max_depth = [None] + list(range(2, 5)),  # Profundidad máxima del árbol
    min_samples_split = list(range(2, 4)),  # Número mínimo de muestras requeridas para dividir un nodo interno
    min_samples_leaf = list(range(2, 4)),  # Número mínimo de muestras requeridas para ser una hoja
    max_features = [None] + [i * .05 for i in list(range(2, 4))],  # Número máximo de características a considerar para la división
    max_leaf_nodes = [None] + list(range(2, 10)),  # Número máximo de nodos hoja
    min_impurity_decrease = [x * .10 for x in list(range(2, 4))]  # Un nodo se dividirá si esta división induce una disminución de la impureza mayor o igual a este valor
)


In [ ]:
# Hiperarametros del modelo KNN
paramknn = dict(n_neighbors= (range(2, 4)),  # Número de vecinos a considerar
          weights=["uniform", "distance"],  # Método de ponderación de los vecinos
          metric= ["euclidean", "manhattan"],  # Métrica de distancia utilizada
          algorithm= ["auto", "ball_tree", "kd_tree", "brute"],  # Algoritmo utilizado para calcular los vecinos más cercanos
          p= [1, 2]  # Parámetro de potencia para la distancia de Minkowski
          )

In [ ]:
# Hiperarametros del modelo Random Forest
param_RF = dict(n_estimators=list(range(1, 100, 25)),
                                    criterion=['gini', 'entropy'],
                                    max_depth=[x for x in list(range(2, 5))] + [None],
                                    min_samples_split=[x for x in list(range(2, 4))],
                                    min_samples_leaf=[x for x in list(range(2, 4))],
                                    max_features=[None] + [i * .05 for i in list(range(2, 4))],
                                    max_leaf_nodes=list(range(2, 10)) + [None],
                                    min_impurity_decrease=[x * .10 for x in list(range(2, 4))],
                                    oob_score=[True,False],
                                    warm_start=[True, False],
                                    class_weight=[None, 'balanced'],
                                    max_samples=[None],)

In [ ]:
# Hiperparametros del modelo Ada Boost
param_adab = dict(n_estimators = range(2,10),
             learning_rate = np.arange(0.1,1,0.1),
             algorithm = ['SAMME.R'])

In [ ]:
# Hiperparametros del modelo GradientBoostingClassifier
param_gradient_boosting = dict(
    n_estimators = list(range(1, 100, 25)),  # Número de árboles de decisión a construir
    learning_rate = [0.001, 0.01, 0.1],  # Tasa de aprendizaje
    max_depth = [3, 4, 5],  # Profundidad máxima de los árboles de decisión
    min_samples_split = list(range(2, 4)),  # Número mínimo de muestras requeridas para dividir un nodo interno
    min_samples_leaf = list(range(1, 3)),  # Número mínimo de muestras requeridas para ser una hoja
    subsample = [0.5, 0.8, 1.0],  # Fracción de muestras utilizadas para ajustar los árboles de decisión
    max_features = [None, 'sqrt', 'log2'],  # Número máximo de características a considerar para la división
    max_leaf_nodes = [None, 5, 10],  # Número máximo de nodos hoja
    min_impurity_decrease = [0.0, 0.1, 0.2]  # Un nodo se dividirá si esta división induce una disminución de la impureza mayor o igual a este valor
)


###Escaladores

In [ ]:
# Lista de nombres de los escaladores
nombres_escaladores = ["standard",
                       "minmax",
                       "robust",
                       "maxabs"]

###Modelos

In [ ]:
# Lista de nombres de los modelos
nombres_modelos = ["Regresión Logística",
                   "Árbol de Decisión",
                   "KNN",
                   "Random Forest",
                   "Ada Boost",
                   "Gradient Boosting Machines"]

**Pipeline**

In [ ]:
# Inicia el tiempo total de ejecución
#total_start_time = time.time()

for model in nombres_modelos:
  # Aplicar modelo correspondiente
  if model == "Regresión Logística":
    modelo = LogisticRegression()
    param = param_logreg
  elif model == "Árbol de Decisión":
    modelo = DecisionTreeClassifier()
    param = param_arbol_decision
  elif model == "KNN":
    modelo = KNeighborsClassifier()
    param = paramknn
  elif model == "Random Forest":
    modelo = RandomForestClassifier()
    param = param_RF
  elif model == "Ada Boost":
    modelo = AdaBoostClassifier()
    param = param_adab
  elif model == "Gradient Boosting Machines":
    modelo = GradientBoostingClassifier()
    param = param_gradient_boosting

  print("")
  print("")
  print("       ",model)

  for scaler_type in nombres_escaladores:
      # Inicia el tiempo de ejecución del modelo actual
      model_start_time = time.time()
      # Aplicar escalado solo a las características numéricas
      if scaler_type == "standard":
          scaler = StandardScaler()
      elif scaler_type == "minmax":
          scaler = MinMaxScaler()
      elif scaler_type == "robust":
          scaler = RobustScaler()
      elif scaler_type == "maxabs":
          scaler = MaxAbsScaler()
      elif scaler_type == "quantile":
          scaler = QuantileTransformer(output_distribution="normal")
      else:
          print("Tipo de escalado no válido.")

      # Escalando datos numericos
      numerical_data_preprocessed = scaler.fit_transform(numerical_data)

      # Creamos un codificador One-Hot
      encoder = OneHotEncoder()

      # Ajustamos y transformamos los datos categóricos utilizando One-Hot Encoding
      encoded_data = encoder.fit_transform(categorical_data)

      # Convertir la matriz sparse resultante en un DataFrame de pandas y nombrar las columnas
      encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(categorical_features))

      # Escalando datos categoricos
      categorical_data_preprocessed = scaler.fit_transform(encoded_df)

      # Concatenando los datos numericos y categoricos escalados
      mixed_data = pd.concat([pd.DataFrame(numerical_data_preprocessed), pd.DataFrame(categorical_data_preprocessed)], axis=1)

      # Asiganando de nuevo los nombres de las columnas
      mixed_data.columns = ["CreditScore", "Age", "Tenure", "Balance",
        "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary",
        "Geography_France", "Geography_Germany", "Geography_Spain",
        "Gender_Female", "Gender_Male"]

      # Obteniendo datos de Entrenamiento y prueba
      train, test, train_y, test_y =  train_test_split(mixed_data, y, test_size=0.30)

      # Finaliza el tiempo de ejecución del modelo actual
      model_end_time = time.time()
      model_execution_time = model_end_time - model_start_time

      modelo_nuevo, best_estimator, score, params = entrenar(param, modelo, train, train_y)
      print(scaler_type)
      print(metricas(train,test,train_y,test_y,modelo_nuevo))
      print(f"Tiempo de ejecución del modelo {model} con el escalamiento {scaler_type}: {model_execution_time:.2f} segundos")
      d = metricas(train,test,train_y,test_y,modelo_nuevo)

      if abs(d["train"] - d["validate"]) < 0.01:  # Si las diferencias son pequeñas
          print("El ajuste del modelo parece apropiado.")
      elif d["train"] > d["validate"]:  # Si la puntuación de entrenamiento es mayor
          print("El modelo está sobreajustado (overfitting).")
      else:  # Si la puntuación de prueba es mayor
          print("El modelo está subajustado (underfitting).")
      print("")



        Regresión Logística
standard
{'train': 0.79, 'validate': 0.793}
Tiempo de ejecución del modelo Regresión Logística con el escalamiento standard: 0.03 segundos
El ajuste del modelo parece apropiado.

minmax
{'train': 0.787, 'validate': 0.804}
Tiempo de ejecución del modelo Regresión Logística con el escalamiento minmax: 0.04 segundos
El modelo está subajustado (underfitting).

robust
{'train': 0.792, 'validate': 0.791}
Tiempo de ejecución del modelo Regresión Logística con el escalamiento robust: 0.03 segundos
El ajuste del modelo parece apropiado.

maxabs
{'train': 0.791, 'validate': 0.794}
Tiempo de ejecución del modelo Regresión Logística con el escalamiento maxabs: 0.03 segundos
El ajuste del modelo parece apropiado.



        Árbol de Decisión
standard
{'train': 0.5, 'validate': 0.5}
Tiempo de ejecución del modelo Árbol de Decisión con el escalamiento standard: 0.03 segundos
El ajuste del modelo parece apropiado.

minmax
{'train': 0.5, 'validate': 0.5}
Tiempo de ejecució